In [1]:
## Imports and settings
import pandas as pd
import numpy as np
import math
import seaborn as sns
import urllib
import glob
import os
from urllib.error import HTTPError

import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
#%matplotlib inline
sns.set_style("darkgrid")

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output


# from tqdm.auto import tqdm
from tqdm.notebook import tnrange, tqdm

tqdm.pandas()  # activate tqdm progressbar for pandas apply

pd.options.mode.chained_assignment = (
    None  # default='warn', remove pandas warning when adding a new column
)
pd.set_option("display.max_columns", None)

from IPython.core.interactiveshell import InteractiveShell


InteractiveShell.ast_node_interactivity = "all"
#%config InlineBackend.figure_format ='svg' #better quality figure figure
np.seterr(divide='ignore', invalid='ignore')

import matplotlib.gridspec as gridspec


import MDAnalysis as mda
import nglview as nv

/Users/thibault/miniconda3/envs/peprmint/lib/python3.7/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [29]:
%run "./00-SETUP.ipynb"

In [3]:
DATASET = pd.read_pickle(f"{WORKDIR}/DATASET_peprmint_d25.pkl")


1          O60493
4          O60493
6          Q9Y2I1
9          Q9Y2I1
14         O60493
            ...  
7391138    Q96J02
7391157    Q9JIR4
7391160    Q9JIR4
7391163    Q9JIR4
7391166    Q9JIR4
Name: uniprot_acc, Length: 1478493, dtype: category
Categories (6282, object): ['A0A024QYT3', 'A0A075B5H6', 'A0A075F932', 'A0A098DRQ4', ..., 'Q9ZVT9', 'S4R1M9', 'V5M2P5', 'W6RTA4']

In [41]:
REBUILD = True
use_uniprot_boundaries = True
use_all_AFmodels = True

import requests
import json
from biopandas.pdb import PandasPdb
from Bio import AlignIO
from urllib.error import URLError
import re
REGEX = re.compile("^(\w+)\|(\w+)\/(\d+)-(\d+)")


EXCLUDE_LIST=["Q54C71","O94827",'Q54C71','Q22070','P39960','Q62077', #PH
             'Q06839', #PX
             ]
EXCLUDE_DOMAIN = ["FYVE"]
    
def fetch_pdb_alfafold(uniprotids, domain):
    nomodels=[]
    withmodels=[]
    outfolder = f"{ALFAFOLDFOLDER}/{domain}/raw"
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
        
    extractedfolder = f"{ALFAFOLDFOLDER}/{domain}/extracted"
    if not os.path.exists(extractedfolder):
        os.makedirs(extractedfolder)
    else:
        if REBUILD == True: #delete extracted files
            files = glob.glob(f"{extractedfolder}/*.pdb")
            for f in files:
                os.remove(f)
    
    jsonfolder = f"{ALFAFOLDFOLDER}/{domain}/json"
    if not os.path.exists(jsonfolder):
        os.makedirs(jsonfolder)

    for uniprot_id in tqdm(uniprotids, desc="Downloading "):
        url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v1.pdb"
        destination = f"{outfolder}/{uniprot_id}.pdb"
        if not os.path.isfile(destination): 
            try:
                urllib.request.urlretrieve(url, destination)
            except urllib.error.HTTPError as err:
                nomodels.append(uniprot_id)
                continue
        withmodels.append(uniprot_id)

    
    print(f"{len(nomodels)} out of {len(uniprotids)} without alfafold2 models ({len(nomodels)/len(uniprotids)*100:.2f}%)")
    return withmodels,nomodels



def get_prosite_boundaries_dict(domain):
    boundaries = {}
    prosite_ids = DOMAIN_PROSITE[domain]
    if type(prosite_ids) != type([]):
        prosite_ids = [prosite_ids]
    for msafile in prosite_ids:
        msafilepath = f"{PROSITEFOLDER}/msa/{msafile}.msa"
        msa = AlignIO.read(msafilepath,'fasta')
        for record in msa:
            seqid = record.id
            match = REGEX.match(seqid)
            if match:
                uniprot_id = match.group(2)
                start = match.group(3)
                end = match.group(4)
                boundaries[uniprot_id] = (int(start),int(end))
    return boundaries

def get_json(uniprot_acc, domain, source='ssf'):
    def request_URL(link, trial=1):
        try:
            response = requests.get(link).text
            return response
        except URLError as e:
            print(e, link)
            if trial >3 :
                print('3rd fail, skipping this one.')
                return None
            else:
                print(f"Trial {trial}, waiting 10s and trying again")
                sleep(10)
                return request_URL(link, trial=trial+1)
            
            
    jsonfolder = f"{ALFAFOLDFOLDER}/{domain}/json"
    if not os.path.exists(jsonfolder):
        os.makedirs(jsonfolder)
        
    jsonfile = f"{jsonfolder}/{uniprot_acc}.json"
    if os.path.isfile(jsonfile):
        f = open(jsonfile)
        interpro = json.load(f)
    else:
        #make the query on ebi/interpro
        response = request_URL(f"https://www.ebi.ac.uk/interpro/api/entry/{source}/protein/reviewed/{uniprot_acc}/?page_size=200")
        if response == None:
            return None
        try:
            interpro = json.loads(response)
        except:
            print(f"no data for {uniprot_acc}.")
            return None
        with open(jsonfile,'w') as out:
            json.dump(interpro, out, indent=2)
            
    return(interpro)
        
    
        

def get_domain_fragment_query(uniprot_acc, domain, boundaries_prosite):
    start_PS,end_PS = boundaries_prosite[uniprot_acc]
    starts_ends = [boundaries_prosite[uniprot_acc]]

    if DOMAIN_INTERPRO_REFINE[domain] == True:
        if domain == "PLA":
            source = 'cathgene3d'
        else:
            source = 'ssf'
        interpro = get_json(uniprot_acc, domain, source)
        if interpro == None:
            return None
        QueryString = None
        
        for result in interpro["results"]:
            if result["metadata"]["accession"] == DOMAIN_INTERPRO[domain]:
                entry_protein_locations = result["proteins"][0]["entry_protein_locations"]
                for entry in entry_protein_locations: #Get the number of truncation in the domain.
                    nfrag = len(entry['fragments'])
                    
                    if domain == 'PLA': #Special case for PLA, we will ignore PROSITE annotation that are actually wrong.
                        frag = entry['fragments'][0] #Get first monomer only
                        s = entry['fragments'][0].get('start')
                        e = entry['fragments'][0].get('end')
                        starts_ends = [[s,e]]
                    else:
                        if nfrag >= 2 and ( entry['fragments'][0].get('start') - 50 <= start_PS <= entry['fragments'][0].get('start')+50) : #if truncated domain AND correspond to the prosite domain
                            print(f"splitting {domain}-{uniprot_acc}")
                            queries = []
                            starts_ends = []
                            for frag in entry['fragments']:
                                s=int(frag.get("start"))
                                e=int(frag.get("end"))
                                starts_ends.append([s,e])
                            if use_uniprot_boundaries == True:
                                starts_ends[0][0] = start_PS
                                starts_ends[-1][-1] = end_PS

                        else: #use prosite fragment
                            starts_ends = [[start_PS, end_PS]]
                    

                QueryString = " or ".join([f"({x} <= residue_number <= {y})" for x,y in starts_ends])
        
    else:
        QueryString = " or ".join([f"({x} <= residue_number <= {y})" for x,y in starts_ends])
    
    return QueryString





## ------- MAIN
domains = DATASET.domain.unique()
domains = ['PLA']

for domain in domains:
    #if domain in EXCLUDE_DOMAIN:
    #    continue
    group = DATASET.query("domain == @domain")
    uniprot_acc_cathpdb = group.query("data_type == 'cathpdb'").uniprot_acc.unique()
    print(f"----- PROCESSING DOMAIN {domain} -----")

    seqs_no_pdb = group[group["pdb"].isnull()].uniprot_acc.unique()
    boundaries_prosite = get_prosite_boundaries_dict(domain)


    if use_all_AFmodels:
        prosite_uniprot_acc = list(boundaries_prosite.keys()) 
        uniprot_acc_cathpdb = [acc for acc in uniprot_acc_cathpdb if acc in prosite_uniprot_acc]

        uniprot_acc_list = prosite_uniprot_acc + uniprot_acc_cathpdb

        seqs_with_model, seqs_without_model=fetch_pdb_alfafold(uniprot_acc_list, 
                                                               domain,
                                                              )
    else:
        seqs_with_model, seqs_without_model=fetch_pdb_alfafold(seqs_no_pdb, 
                                                               domain,
                                                              )


    for uniprot_id in tqdm(seqs_with_model, desc = "processing"):
        if uniprot_id in EXCLUDE_LIST:
            continue

        pdbfile =  f"{ALFAFOLDFOLDER}/{domain}/raw/{uniprot_id}.pdb"


        # structure = PDBParser().get_structure('uniprot_id',)    

        if os.path.isfile(pdbfile) and REBUILD == False:
            #skip the file if already exist
            continue


        query = get_domain_fragment_query(uniprot_id, domain, boundaries_prosite)
        if query == None:
            continue
        ppdb = PandasPdb().read_pdb(pdbfile)
        ppdb.df["ATOM"] = ppdb.df["ATOM"].query(f"{query}")
        ppdb.to_pdb(f"{ALFAFOLDFOLDER}/{domain}/extracted/{uniprot_id}.pdb")

----- PROCESSING DOMAIN PLA -----



512 out of 570 without alfafold2 models (89.82%)


no data for Q9H295.
no data for Q8WXA2.
no data for Q9U256.
no data for Q8BT42.
no data for Q8N271.
no data for Q5TA77.
no data for Q5TA76.
no data for Q8TEX9.
no data for Q71RC9.
no data for Q6ZRS4.



In [37]:
get_domain_fragment_query("Q9Z0L3", 'PLA', boundaries_prosite)

'(314 <= residue_number <= 435)'

urllib.error.URLError

In [19]:
from Bio import AlignIO
import re
REGEX = re.compile("^(\w+)\|(\w+)\/(\d+)-(\d+)")         


all_uniprot_acc = []
for domain in DATASET.domain.unique():
    prosite_ids = DOMAIN_PROSITE[domain]
    if type(prosite_ids) != type([]):
        prosite_ids = [prosite_ids]
    for msafile in prosite_ids:
        msafilepath = f"{PROSITEFOLDER}/msa/{msafile}.msa"
        msa = AlignIO.read(msafilepath,'fasta')
        print(dir(msa))


['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_append', '_get_per_column_annotations', '_per_col_annotations', '_records', '_set_per_column_annotations', '_str_line', 'add_sequence', 'annotations', 'append', 'column_annotations', 'extend', 'format', 'get_alignment_length', 'sort', 'substitutions']


ZeroDivisionError: division by zero

In [60]:
def get_domain_fragment_query(uniprot_acc, domain, boundaries_prosite):
    start_PS,end_PS = boundaries_prosite[uniprot_acc]
    starts_ends = [boundaries_prosite[uniprot_acc]]

    if DOMAIN_INTERPRO_REFINE[domain] == True:
        interpro = get_json(uniprot_acc, domain)
        
        for result in interpro["results"]:
            if result["metadata"]["accession"] == DOMAIN_INTERPRO[domain]:
                entry_protein_locations = result["proteins"][0]["entry_protein_locations"]
                

                for entry in entry_protein_locations: #Get the number of truncation in the domain.
                    nfrag = len(entry['fragments'])
                        
                    if nfrag >= 2 and ( entry['fragments'][0].get('start') - 50 <= start_PS <= entry['fragments'][0].get('start')+50) : #if truncated domain AND correspond to the prosite domain
                        queries = []
                        starts_ends = []
                        for frag in entry['fragments']:
                            print(frag)
                            s=int(frag.get("start"))
                            e=int(frag.get("end"))
                            starts_ends.append([s,e])
                        if use_uniprot_boundaries == True:
                            starts_ends[0][0] = start_PS
                            starts_ends[-1][-1] = end_PS
                            
                    else: #use prosite fragment
                        starts_ends = [[start_PS, end_PS]]
                    

                QueryString = " or ".join([f"({x} <= residue_number <= {y})" for x,y in starts_ends])
        
    else:
        QueryString = " or ".join([f"({x} <= residue_number <= {y})" for x,y in starts_ends])
    return QueryString



boundaries = get_prosite_boundaries_dict("PH")
query = get_domain_fragment_query('Q55E26', "PH", boundaries)
#query = get_domain_fragment_query('F1LXF1', "PH", boundaries)
query

{'start': 853, 'end': 942, 'dc-status': 'C_TERMINAL_DISC'}
{'start': 994, 'end': 1026, 'dc-status': 'N_TERMINAL_DISC'}


'(876 <= residue_number <= 942) or (994 <= residue_number <= 1026)'

In [42]:
query[0]["fragments"][0]['start']

188

read the MSA and get the position.

In [137]:
from Bio import AlignIO
import re
regex = re.compile("^(\w+)\|(\w+)\/(\d+)-(\d+)")
boundaries = {}

prosite_ids = DOMAIN_PROSITE[domain]
if type(prosite_ids) != type([]):
    prosite_ids = [prosite_ids]
for msafile in prosite_ids:
    msafilepath = f"{PROSITEFOLDER}/msa/{msafile}.msa"
    msa = AlignIO.read(msafilepath,'fasta')
    for record in msa:
        seqid = record.id
        match = regex.match(seqid)
        if match:
            uniprot_id = match.group(2)
            start = match.group(3)
            end = match.group(4)
            boundaries[uniprot_id] = (int(start),int(end))

In [17]:
seqs_no_pdb

['Q08748', 'Q9ESD7', 'O00750', 'Q7ZWU7', 'Q9Y6V0', ..., 'A5D6R3', 'Q4R6L3', 'Q8BTI9', 'O77793', 'A0FGR9']
Length: 132
Categories (132, object): ['Q08748', 'Q9ESD7', 'O00750', 'Q7ZWU7', ..., 'Q4R6L3', 'Q8BTI9', 'O77793', 'A0FGR9']

('821', '921')

Reading alfafold pdbs and saving only PH domains.

In [20]:
from Bio.PDB import PDBParser


for uniprot_id in tqdm(seqs_with_model):
    start,end = boundaries[uniprot_id]
    pdbfile =  f"{ALFAFOLDFOLDER}/{domain}/raw/{uniprot_id}.pdb"
    # structure = PDBParser().get_structure('uniprot_id',)    

    from biopandas.pdb import PandasPdb
    ppdb = PandasPdb().read_pdb(pdbfile)

    ppdb.df["ATOM"] = ppdb.df["ATOM"].query("@start <= residue_number <= @end")
    ppdb.to_pdb(f"{ALFAFOLDFOLDER}/{domain}/{uniprot_id}.pdb")

In [23]:
def fetch_json_information(uniprot_acc):
    response = requests.get(f"https://www.ebi.ac.uk/interpro/api/entry/ssf/protein/reviewed/{uniprot_acc}/?page_size=200").text
    interpro = json.loads(response)

In [133]:
%%time
import requests
import json
domain='PH'
uniprot_acc = "F1LXF1"
response = requests.get(f"https://www.ebi.ac.uk/interpro/api/entry/ssf/protein/reviewed/{uniprot_acc}/?page_size=200").text
interpro = json.loads(response)

CPU times: user 15.8 ms, sys: 10.7 ms, total: 26.5 ms
Wall time: 180 ms


In [139]:
%%time

use_uniprot_boundaries = True

start_PS,end_PS = boundaries[uniprot_acc]

for result in interpro["results"]:
    if result["metadata"]["accession"] == DOMAIN_INTERPRO[domain]:
        fragments = result["proteins"][0]["entry_protein_locations"][0]["fragments"]
        if len(fragments) >= 2:
            starts_ends = []
            queries = []
            for frag in fragments:
                s=int(frag.get("start"))
                e=int(frag.get("end"))
                starts_ends.append([s,e])
#                queries.append(f"({s} <= residue_number <= {e})")
            if use_uniprot_boundaries == True:
                starts_ends[0][0] = start_PS
                starts_ends[-1][-1] = end_PS
        else: #use prosite fragment
            starts_ends = [start_PS, end_PS]
            
        QueryString = " or ".join([f"({x} <= residue_number <= {y})" for x,y in starts_ends])
        
            
            
print(QueryString)

(499 <= residue_number <= 566) or (624 <= residue_number <= 657)
CPU times: user 273 µs, sys: 103 µs, total: 376 µs
Wall time: 335 µs


In [65]:
get_domain_fragment_query("Q54C71", domain, boundaries)

'(601 <= residue_number <= 822)'

In [173]:
def get_json(uniprot_acc, domain):
    jsonfolder = f"{ALFAFOLDFOLDER}/{domain}/json"
    if not os.path.exists(jsonfolder):
        os.makedirs(jsonfolder)
        
    jsonfile = f"{jsonfolder}/{uniprot_acc}.json"
    if os.path.isfile(jsonfile):
        f = open(jsonfile)
        interpro = json.load(f)
    else:
        #make the query on ebi/interpro
        response = requests.get(f"https://www.ebi.ac.uk/interpro/api/entry/ssf/protein/reviewed/{uniprot_acc}/?page_size=200").text
        interpro = json.loads(response)
        with open(jsonfile,'w') as out:
            json.dump(interpro, out, indent=2)
            
    return(interpro)
                    

{'count': 4,
 'next': None,
 'previous': None,
 'results': [{'metadata': {'accession': 'SSF48403',
    'name': 'Ankyrin repeat',
    'source_database': 'ssf',
    'type': 'homologous_superfamily',
    'integrated': 'IPR036770',
    'member_databases': None,
    'go_terms': None},
   'proteins': [{'accession': 'q8cgu4',
     'protein_length': 1186,
     'source_database': 'reviewed',
     'organism': '10116',
     'entry_protein_locations': [{'fragments': [{'start': 1051,
         'end': 1143,
         'dc-status': 'CONTINUOUS'}],
       'model': '0047382',
       'score': 4.98e-16}]}]},
  {'metadata': {'accession': 'SSF50729',
    'name': 'PH domain-like',
    'source_database': 'ssf',
    'type': 'homologous_superfamily',
    'integrated': None,
    'member_databases': None,
    'go_terms': None},
   'proteins': [{'accession': 'q8cgu4',
     'protein_length': 1186,
     'source_database': 'reviewed',
     'organism': '10116',
     'entry_protein_locations': [{'fragments': [{'start': 6

In [43]:
DATASET.groupby("cathpdb")

1           [Early endosome, Cytoplasmic vesicle, Phagosome]
4           [Early endosome, Cytoplasmic vesicle, Phagosome]
6          [Cell membrane, Cytoplasm, Early endosome, Rec...
9          [Cell membrane, Cytoplasm, Early endosome, Rec...
14          [Early endosome, Cytoplasmic vesicle, Phagosome]
                                 ...                        
7395878                                           [Secreted]
7395879                                           [Secreted]
7395880                                           [Secreted]
7395881                                           [Secreted]
7395882                                           [Secreted]
Name: location, Length: 1519907, dtype: object